In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
import cv2
import time
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf
!pip install tensorflow-addons==0.16.1
import tensorflow_addons as tfa
from keras.models import model_from_json
# from utils_gen import gen_paths_img_dm, gen_var_from_paths
# from utils_imgproc import norm_by_imagenet
from tensorflow.keras import models,layers
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.models import Model,Sequential
from scipy.io import loadmat
from tqdm import tqdm
from utils_gen import gen_density_map_gaussian
import scipy.io as io
from PIL import Image
import PIL
import random
%matplotlib inline
plt.ioff()

# Settings
net = 'CSRNet'
dataset = "A"
# Generate paths of (train, test) x (img, dm)
# Generate paths of (train, test) x (img, dm)

train_img_paths=['/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_10.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_128.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_133.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_149.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_166.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_203.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_208.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_206.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_235.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_254.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_39.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_41.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_43.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_44.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_5.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_65.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_67.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_61.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_68.jpg',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/images/IMG_83.jpg']
test_img_paths=['/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_20.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_19.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_21.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_29.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_32.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_38.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_39.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_43.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_57.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_7.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_11.jpg',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_14.jpg'
]
train_dm_paths=['/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_10.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_128.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_133.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_149.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_166.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_203.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_208.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_206.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_235.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_254.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_39.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_41.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_43.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_44.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_5.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_65.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_67.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_61.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_68.h5',
                '/content/gdrive/MyDrive/data/ShanghaiTech/train_data/ground-truth-h5/IMG_83.h5']


test_dm_paths=['/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_20.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_19.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_21.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_29.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_32.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_38.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_39.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_43.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_57.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_7.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_11.h5',
'/content/gdrive/MyDrive/data/ShanghaiTech/test_data/ground-truth-h5/IMG_14.h5'
]

target_type=tf.dtypes.float32
def load_image(path):
    X=[]
    for adress in path:
      img=cv2.imread(adress)
      #imgnp=np.array(img)
      #print(imgnp.shape)
      image = np.array(Image.open(adress, "r").convert("RGB"))
      image = image / 255.0
      image[:, :, 0] = (image[:, :, 0] - 0.485) / 0.229
      image[:, :, 1] = (image[:, :, 1] - 0.456) / 0.224
      image[:, :, 2] = (image[:, :, 2] - 0.406) / 0.225
      #print(image.shape)
      image2=tf.image.convert_image_dtype(image, target_type)
      #print(image2.shape)
      X.append(image2)
    return np.array(X).reshape(-1,683,1024,3)
y_train=[]
for img_path in train_img_paths:
    img_ori = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    pts = loadmat(img_path.replace('.jpg', '.mat').replace('images', 'ground-truth').replace('IMG_', 'GT_IMG_'))
    img = cv2.imread(img_path)
    
    sigma = 4 
    k = np.zeros((img.shape[0], img.shape[1]))
    gt = pts["image_info"][0, 0][0, 0][0]
    for i in range(len(gt)):
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            k[int(gt[i][1]), int(gt[i][0])] = 1

    DM = gen_density_map_gaussian(k, gt, sigma=sigma)
    y_train.append(np.sum(DM))
y_test=[]
for img_path in test_img_paths:
    img_ori = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    pts = loadmat(img_path.replace('.jpg', '.mat').replace('images', 'ground-truth').replace('IMG_', 'GT_IMG_'))
    img = cv2.imread(img_path)
    
    sigma = 4  
    k = np.zeros((img.shape[0], img.shape[1]))
    gt = pts["image_info"][0, 0][0, 0][0]
    for i in range(len(gt)):
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            k[int(gt[i][1]), int(gt[i][0])] = 1

    DM = gen_density_map_gaussian(k, gt, sigma=sigma)
    y_test.append(np.sum(DM))
print(len(y_train))
print(len(y_test))
print(y_train)
print(y_test)
train_size=10
X_train=load_image(train_img_paths[:train_size])
print(X_train.shape)
X_val=load_image(train_img_paths[train_size:])
print(X_val.shape)
y_train=np.array([919.9997, 1455.9999, 827.00006, 2071.9998, 899.00024, 1956.0007, 536.0, 311.00006, 189.99992, 581.0])
print(y_train.shape)
y_val=np.array([411.0, 402.0, 239.99988, 280.00018, 584.99976, 426.99985, 163.00005, 88.00001, 199.00006, 588.0001])
print(y_val.shape)
X_test=load_image(test_img_paths)
print(X_test.shape)
y_test=np.array(y_test)
print(y_test.shape)




def CSRNet():

    
    input_flow = Input(shape=(683,1024,3))
    dilated_conv_kernel_initializer = tf.keras.initializers.RandomNormal(stddev=0.01)

    # front-end
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(input_flow)
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    #x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    #x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    #x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x=tf.keras.layers.UpSampling2D(size=(8,8))(x)
    # back-end
    #x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    #x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    #x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    output_flow = Conv2D(1, 1, strides=(1, 1), padding='same', activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)

    model = Model(inputs=input_flow, outputs=output_flow)

    front_end = VGG16(weights='imagenet', include_top=False)

    weights_front_end = []
    for layer in front_end.layers:
        if 'conv' in layer.name:
            weights_front_end.append(layer.get_weights())
    counter_conv = 0
    for i in range(len(front_end.layers)):
        if counter_conv >= 7:
            break
        if 'conv' in model.layers[i].name:
            model.layers[i].set_weights(weights_front_end[counter_conv])
            counter_conv += 1
    return model
sgd = SGD(lr = 1e-4, momentum = 0.7, nesterov=True)
rms = RMSprop(lr=1e-4, momentum=0.7, decay=0.0001)
nadam = Nadam(lr=1e-2)

optimizer = Adam(lr=1e-5)
loss=tf.keras.losses.mse

batch_size=1

model = CSRNet()
model.compile(optimizer=optimizer, loss=loss, metrics=[tf.keras.metrics.MeanAbsoluteError()])
#model.build(input_shape=(None, None, None, 3))
#print('train baseline: ', np.mean( np.sum(train_labels[:train_size], axis=(1,2,3))))
#print('val baseline: ', np.mean( np.sum(train_labels[320:], axis=(1,2,3))))
model.summary()
model.save('Crowd_CNN.h5')
model.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=1,epochs=20)

import numpy as np
import cv2
import scipy
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import spatial

#model = load_model('crowd_FCN.h5')

def gen_density_map_gaussian(im, points, sigma=4):
    """
    func: generate the density map
    """
    density_map = np.zeros(im.shape[:2], dtype=np.float32)
    h, w = density_map.shape[:2]
    num_gt = np.squeeze(points).shape[0]
    if num_gt == 0:
        return density_map
    if sigma == 4:
        # Adaptive sigma in CSRNet.
        leafsize = 2048
        tree = scipy.spatial.KDTree(points.copy(), leafsize=leafsize)
        distances, _ = tree.query(points, k=4)
    for idx_p, p in enumerate(points):
        p = np.round(p).astype(int)
        p[0], p[1] = min(h-1, p[1]), min(w-1, p[0])
        gaussian_radius = sigma * 2 - 1
        if sigma == 4:
            # Adaptive sigma in CSRNet.
            sigma = max(int(np.sum(distances[idx_p][1:4]) * 0.1), 1)
            gaussian_radius = sigma * 3
        gaussian_map = np.multiply(
            cv2.getGaussianKernel(int(gaussian_radius*2+1), sigma),
            cv2.getGaussianKernel(int(gaussian_radius*2+1), sigma).T
        )
        x_left, x_right, y_up, y_down = 0, gaussian_map.shape[1], 0, gaussian_map.shape[0]
        # cut the gaussian kernel
        if p[1] < gaussian_radius:
            x_left = gaussian_radius - p[1]
        if p[0] < gaussian_radius:
            y_up = gaussian_radius - p[0]
        if p[1] + gaussian_radius >= w:
            x_right = gaussian_map.shape[1] - (gaussian_radius + p[1] - w) - 1
        if p[0] + gaussian_radius >= h:
            y_down = gaussian_map.shape[0] - (gaussian_radius + p[0] - h) - 1
        gaussian_map = gaussian_map[y_up:y_down, x_left:x_right]
        if np.sum(gaussian_map):
            gaussian_map = gaussian_map / np.sum(gaussian_map)
        density_map[
            max(0, p[0]-gaussian_radius):min(h, p[0]+gaussian_radius+1),
            max(0, p[1]-gaussian_radius):min(w, p[1]+gaussian_radius+1)
        ] += gaussian_map
    density_map = density_map / (np.sum(density_map / num_gt))
    return density_map

img_paths=['/content/gdrive/MyDrive/data/ShanghaiTech/test_data/images/IMG_10.jpg']
for  img_path in  img_paths:
    img = cv2.imread(img_path)
    img2 = cv2.resize(img,(1024,683))
    img2=np.array(img2)
    img=np.array(img)
    pred = model.predict(np.expand_dims(img2, axis=0))
    print(pred)
    pred=np.squeeze(pred, axis=0)
    new_pred=np.squeeze(pred,axis=2)
    print(new_pred.shape)
    plt.imshow(new_pred,cmap=plt.cm.jet)
    fg, (ax_x_ori, ax_y) = plt.subplots(1, 2, figsize=(20, 4))
    ax_x_ori.imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    ax_x_ori.set_title('Original Image')
    pts = loadmat(img_path.replace('.jpg', '.mat').replace('images', 'ground-truth').replace('IMG_', 'GT_IMG_'))
    k = np.zeros((img.shape[0], img.shape[1]))
    gt = pts["image_info"][0, 0][0, 0][0]
    for i in range(len(gt)):
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            k[int(gt[i][1]), int(gt[i][0])] = 1

    DM = gen_density_map_gaussian(k, gt, sigma=4)

    ax_y.imshow(np.squeeze(DM), cmap=plt.cm.jet)
    ax_y.set_title('DM -- '+str(np.sum(DM)))

    #ax_pred.imshow(pred, cmap=plt.cm.jet)
    #ax_pred.set_title('Prediction: ' + str(np.sum(pred)))
    plt.show()


